In [1]:
import numpy as np
import pandas as pd
import cv2
from sldata import SessionData

### Load Bonsai Data (Now with SLEAP) 

In [2]:
mouse_id = "7004"
session_id = "v4"
experiment = "clickbait-visual"

print("=== Testing SessionData Class ===")
print(f"Loading data for {mouse_id}_{session_id}...")

data = SessionData(
    mouse_id=mouse_id,
    session_id=session_id,
    experiment=experiment,
    min_spikes=100,
    verbose=True)

=== Testing SessionData Class ===
Loading data for 7004_v4...
Successfully loaded spike_times from S:\clickbait-visual\kilosorted\7004\v4\spike_times.npy
Successfully loaded spike_templates from S:\clickbait-visual\kilosorted\7004\v4\spike_templates.npy
Successfully loaded templates from S:\clickbait-visual\kilosorted\7004\v4\templates.npy
Successfully loaded sniff from S:\clickbait-visual\preprocessed\7004\v4\sniff.npy
Loaded data: ['spike_times', 'spike_templates', 'templates', 'sniff']
Successfully loaded events from S:\clickbait-visual\bonsai_align\7004\v4\events.csv
Video properties: 888x1968, 30.0 FPS, 109108 frames
Found 26949 peaks in sniff signal
Peak times range: 1.4 - 122288.8 ms
Loaded 32 clusters for 7004_v4
Loaded sniff data: 3668950 samples
Found 26949 sniff events


In [3]:
data.events[-5:]

,trial_number,timestamp,poke_left,poke_right,centroid_x,centroid_y,target_cell,iti,water_left,water_right,reward_state,click,speed,direction,drinking,timestamp_ms
109088,28,2025-04-07 14:25:10.574208000,False,False,709,304,NaN,False,True,False,False,False,4.123106,1.325818,False,3663418
109089,28,2025-04-07 14:25:10.665920000,False,False,708,307,NaN,False,True,False,False,False,3.162278,0.000046,False,3663451
109090,28,2025-04-07 14:25:10.753510400,False,False,745,324,NaN,False,True,False,False,False,40.718548,0.430693,False,3663485
109091,28,2025-04-07 14:25:10.844160000,False,False,745,320,NaN,False,True,False,False,False,4.000000,1.570796,False,3663518
109092,28,2025-04-07 14:25:10.955686400,False,False,745,313,NaN,False,True,False,False,False,7.000000,1.570796,False,3663551


### Visualize SLEAP Tracking

In [8]:
# Visualization options
scale = 0.5  # Scaling factor for preview animation
save = False # Toggle preview
video = True  # Toggle video background
bonsai_centroid = False  # Centroid from realtime tracking

# SLEAP Nodes
nose = data.events[['nose.x', 'nose.y']].to_numpy().astype(np.int16)
cent = data.events[['centroid.x', 'centroid.y']].to_numpy().astype(np.int16)
base = data.events[['tailbase.x', 'tailbase.y']].to_numpy().astype(np.int16)
bon_cent = data.events[['bonsai_centroid_x', 'bonsai_centroid_y']].to_numpy().astype(np.int16)


for ii in range(0, data.total_frames):
    if video:
        data.video.set(cv2.CAP_PROP_POS_FRAMES, ii + data.video_offset)  # Get current video frame
        ret, canvas = data.video.read()
    else:
        canvas = np.zeros((data.height,data.width,3), dtype=np.uint8)  # Clear canvas
    
    # Draw Bonsai centroid
    if bonsai_centroid:
        cv2.circle(img=canvas, center=bon_cent[ii], radius=12, color=(128,128,128), thickness=-1, lineType=cv2.LINE_AA)

    # Draw SLEAP nodes
    cv2.circle(img=canvas, center=nose[ii], radius=5, color=(255,0,0), thickness=-1, lineType=cv2.LINE_AA)  # Nose
    cv2.circle(img=canvas, center=cent[ii], radius=5, color=(0,255,0), thickness=-1, lineType=cv2.LINE_AA)  # Centroid
    cv2.circle(img=canvas, center=base[ii], radius=5, color=(0,0,255), thickness=-1, lineType=cv2.LINE_AA)  # Tail base

    canvas = cv2.resize(canvas, dsize=(int(data.width*scale), int(data.height*scale)))
    canvas = cv2.rotate(canvas, cv2.ROTATE_90_CLOCKWISE) 

    if save:
        cv2.imwrite(f"S:/track-test-video/{mouse_id}_{session_id}_tracking_{ii}.png", canvas)
    else:
        cv2.imshow('SLEAP Test', canvas)
        # Wait for key press with timeout
        key = cv2.waitKey(16) & 0xFF
        # Break on 'q' key or ESC key
        if key == ord('q') or key == 27:
            break

cv2.waitKey(0)
cv2.destroyAllWindows()